In [ ]:
from datetime import datetime
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_Local
from matplotlib.patches import Rectangle
from scipy import stats
from scipy.stats import kruskal
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from spreg import OLS, ML_Lag, ML_Error
import geopandas as gpd
import libpysal
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
merged_df = pd.read_csv(r"C:\Users\16643\Desktop\data\trip_merged_dataset.csv", sep=',')

In [ ]:
# =============================================================================
# Data Cleaning & Mapping: Income Groups and Main Mode Categories
# =============================================================================

income_mapping = {
    1: "Less than £25,000",
    2: "£25,000 to £49,999", 
    3: "£50,000 and over"
}


mode_group_map = {
    1: "Active Travel", 2: "Active Travel",           # Walking, Cycling
    3: "Private Vehicle", 4: "Private Vehicle",       # Car driver/passenger
    5: "Private Vehicle", 6: "Private Vehicle",       # Motorcycle, Other private
    7: "Bus", 8: "Bus", 9: "Bus",                     # Local bus, Non-local bus, Coach
    10: "Rail / Underground", 11: "Rail / Underground", # Surface rail, Underground
    12: "Taxi / Other Public", 13: "Taxi / Other Public" # Taxi, Other public
}


merged_df["HHIncomeGroup"] = merged_df["HHIncome2002_B02ID"].map(income_mapping)
merged_df['MainMode_Group'] = merged_df['MainMode_B04ID'].map(mode_group_map)

merged_df_clean = merged_df[
    (merged_df['HHIncomeGroup'].notna()) &
    (merged_df['MainMode_Group'].notna()) &
    (merged_df['TripTotalTime'] >= 1) & 
    (merged_df['TripTotalTime'] <= 180)
].copy()


income_order = ['Less than £25,000', '£25,000 to £49,999', '£50,000 and over']
merged_df_clean['HHIncomeGroup'] = pd.Categorical(merged_df_clean['HHIncomeGroup'], 
                                                  categories=income_order, ordered=True)



In [ ]:
# =============================================================================
# Figure: Mode Share by Income & Trip Duration Distribution
# =============================================================================

mpl.rcParams['font.family'] = 'serif' 
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun']
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.unicode_minus'] = False  

mpl.rcParams['text.color'] = 'black'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['axes.titlecolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['ytick.color'] = 'black'

sns.set_theme(style='whitegrid', rc={
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun'],
    'font.size': 10
})

fig, axes = plt.subplots(2, 1, figsize=(9, 12))  

ax1 = axes[0]
mode_prop = merged_df_clean.groupby(['HHIncomeGroup', 'MainMode_Group']).size().unstack(fill_value=0)
mode_prop_pct = mode_prop.div(mode_prop.sum(axis=1), axis=0) * 100

mode_order = ['Active Travel', 'Bus', 'Taxi / Other Public', 'Rail / Underground', 'Private Vehicle']
mode_prop_pct = mode_prop_pct.reindex(columns=mode_order, fill_value=0)

colors = ['#34495e', '#5d6d7e', '#85929e', '#aeb6bf', '#d5dbdb']
mode_prop_pct.plot(kind='bar', stacked=True, ax=ax1, color=colors, width=0.65, 
                   edgecolor='white', linewidth=0.5)

ax1.set_title('Transport Mode Choice by Income Level', 
              fontweight='normal', fontsize=15, pad=20)
ax1.set_xlabel('Household Income Group', fontweight='normal')
ax1.set_ylabel('Percentage of Trips (%)', fontweight='normal')
ax1.legend(title='Transport Mode\n(Speed: Slow → Fast)', 
           bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9,
           title_fontsize=9, frameon=True, fancybox=True, shadow=False)

_wrap_map = {
    'Less than £25,000': 'Less than\n£25,000',
    '£25,000 to £49,999': '£25,000 to\n£49,999',
    '£50,000 and over': '£50,000 and\nover'
}
_xtlabs = [t.get_text() for t in ax1.get_xticklabels()]
ax1.set_xticklabels([_wrap_map.get(t, t) for t in _xtlabs], rotation=0, ha='center')

for i, income in enumerate(income_order):
    active_bus_pct = mode_prop_pct.loc[income, ['Active Travel', 'Bus']].sum()
    private_pct = mode_prop_pct.loc[income, 'Private Vehicle']
    
    ax1.text(i, 10, f'{active_bus_pct:.0f}%\nSlow modes', ha='center', va='bottom', 
             fontweight='normal', color='white', fontsize=9,
             bbox=dict(boxstyle="round,pad=0.3", facecolor='#2c3e50', alpha=0.9))
    
    ax1.text(i, 65, f'{private_pct:.0f}% car', ha='center', va='top',
             fontweight='normal', color='white', fontsize=9,
             bbox=dict(boxstyle="round,pad=0.3", facecolor='#27ae60', alpha=0.9))

ax2 = axes[1]

contrast_data = merged_df_clean[
    merged_df_clean['MainMode_Group'].isin(['Active Travel', 'Bus', 'Private Vehicle'])
].copy()

contrast_data['Mode_Category'] = contrast_data['MainMode_Group'].map({
    'Private Vehicle': 'Private Car\n(High income)',
    'Active Travel': 'Walking/Cycling\n(Low income)',
    'Bus': 'Bus\n(Low income)'
})

speed_order = ['Private Car\n(High income)', 'Walking/Cycling\n(Low income)', 'Bus\n(Low income)']

palette = {
    'Private Car\n(High income)': '#2c3e50',
    'Walking/Cycling\n(Low income)': '#7f8c8d',
    'Bus\n(Low income)': '#95a5a6'
}

sns.boxplot(data=contrast_data, x='Mode_Category', y='TripTotalTime', ax=ax2,
            order=speed_order, palette=palette, width=0.6)

ax2.set_title('Travel Time Distribution by Transport Mode', 
              fontweight='normal', fontsize=15, pad=20)
ax2.set_xlabel('Transport Mode', fontweight='normal')
ax2.set_ylabel('Trip Duration (minutes)', fontweight='normal')
ax2.tick_params(axis='x', labelsize=10)
ax2.grid(True, alpha=0.3)

mode_stats = []
for i, category in enumerate(speed_order):
    subset = contrast_data[contrast_data['Mode_Category'] == category]
    if len(subset) > 0:
        mean_time = subset['TripTotalTime'].mean()
        mode_stats.append((category, mean_time))
        
        ax2.text(i, ax2.get_ylim()[1] * 0.92, 
                 f'Mean: {mean_time:.1f} min', 
                 ha='center', va='top', fontweight='normal', fontsize=10,
                 bbox=dict(boxstyle="round,pad=0.3", facecolor='#ecf0f1', 
                           edgecolor='#bdc3c7', alpha=0.95))

plt.tight_layout(h_pad=7)

fig.patch.set_facecolor('white')
for ax in axes:
    ax.set_facecolor('#f8f9fa')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('#95a5a6')
    ax.spines['bottom'].set_color('#95a5a6')

plt.show()


In [ ]:
# =============================================================================
# Dataset Assembly: Load PTAL/IMD, Map LSOA11→LSOA21, Clean PTAL Grades, Merge Geo
# =============================================================================

ptal_df = pd.read_csv(r"C:\Users\16643\Desktop\data\LSOA_aggregated_PTAL_stats_2023.csv")
ptal_df['LSOA21CD'] = ptal_df['LSOA21CD'].astype(str).str.strip()

imd_df = pd.read_csv(r"C:\Users\16643\Desktop\data\london_imd.csv")
imd_df = imd_df.rename(columns={
    'LSOA code (2011)': 'LSOA11CD',
    'Index of Multiple Deprivation (IMD) quintile rebased for London': 'IMD_quintile_london'
})

crosswalk_df = pd.read_csv(
    r"C:\Users\16643\Desktop\data\LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Exact_Fit_Lookup_for_EW_(V3).csv"
)
crosswalk_df = crosswalk_df[['LSOA11CD', 'LSOA21CD']]

imd_2021 = imd_df.merge(crosswalk_df, on='LSOA11CD', how='left')


imd_cols = [
    'LSOA21CD',
    'IMD_quintile_london',
    'Income_london_decile',
    'employment_london_decile',
    'edu_london_decile',
    'health_london_decile',
    'crime_london_decile',
    'barriers_london_decile',
    'livingEnv_london_decile'
]

imd_2021 = imd_2021[imd_cols]

imd_2021 = imd_2021.groupby('LSOA21CD', as_index=False).mean()

lsoa21_gdf = gpd.read_file(r"C:\Users\16643\Desktop\data\LSOA2021\LSOA_2021_EW_BSC_V4.shp")
lsoa21_gdf['LSOA21CD'] = lsoa21_gdf['LSOA21CD'].astype(str).str.strip()
imd_2021['LSOA21CD'] = imd_2021['LSOA21CD'].astype(str).str.strip()

lsoa21_gdf = lsoa21_gdf[lsoa21_gdf['LSOA21CD'].isin(ptal_df['LSOA21CD'])].copy()

merged_gdf = lsoa21_gdf.merge(
    ptal_df[['LSOA21CD', 'MEAN_PTAL_']],
    on='LSOA21CD',
    how='left'
)


merged_gdf = merged_gdf.merge(imd_2021, on='LSOA21CD', how='left')


def clean_ptal(val):
    if pd.isna(val):
        return np.nan
    
    val_str = str(val).strip().lower()
    

    if 'b' in val_str:
        try:
            base_num = float(val_str.replace('b', ''))
            return base_num + 0.5
        except:
            return np.nan
    
 
    if 'a' in val_str:
        try:
            base_num = float(val_str.replace('a', ''))
            return 0.5 if base_num == 1 else base_num
        except:
            return np.nan
    
  
    try:
        return float(val_str)
    except:
        return np.nan


merged_gdf['MEAN_PTAL_CLEAN'] = merged_gdf['MEAN_PTAL_'].apply(clean_ptal)

merged_gdf_clean = merged_gdf.dropna(subset=['MEAN_PTAL_CLEAN']).copy()

print(f"The final number of LSOAs for analysis: {len(merged_gdf_clean)}")

merged_gdf_clean['IMD_quintile_london'] = merged_gdf_clean['IMD_quintile_london'].round().astype(int)





In [ ]:
# =============================================================================
# Figure: Public Transport Accessibility (PTAL) and Deprivation (IMD) Maps
# =============================================================================

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun']
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid', rc={'font.family': 'serif', 'font.size': 12})

# 1: PTAL map
fig1, ax1 = plt.subplots(1, 1, figsize=(10, 10))
merged_gdf_clean.plot(
    column='MEAN_PTAL_CLEAN',
    cmap='YlGnBu',
    legend=True,
    legend_kwds={
        'label': "Public Transport Accessibility Level (PTAL)",
        'orientation': "vertical",
        'shrink': 0.5
    },
    edgecolor='grey',
    linewidth=0.3,
    ax=ax1,
    missing_kwds={'color': 'lightgrey'}
)
ax1.set_title(
    "Public Transport Accessibility Level (PTAL) in London by LSOA",
    fontsize=15, pad=20
)
ax1.axis('off')
plt.tight_layout()
plt.show()

# 2: IMD map
fig2, ax2 = plt.subplots(1, 1, figsize=(10, 10))
merged_gdf_clean.plot(
    column='IMD_quintile_london',
    cmap='OrRd',
    linewidth=0.2,
    edgecolor='grey',
    legend=True,
    legend_kwds={
        'label': "Index of Multiple Deprivation (IMD) Quintile",
        'orientation': "vertical",
        'shrink': 0.5
    },
    ax=ax2,
    missing_kwds={'color': 'lightgrey'}
)
ax2.set_title(
    "Index of Multiple Deprivation (IMD) Quintiles in London by LSOA",
    fontsize=15, pad=20
)
ax2.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# Figure: London Transport–Deprivation Spatial Patterns and Correlation Analysis
# =============================================================================

if 'IMD_quintile_london' not in merged_gdf_clean.columns:
    raise KeyError("IMD_quintile_london column missing!")
if 'MEAN_PTAL_CLEAN' not in merged_gdf_clean.columns:
    raise KeyError("MEAN_PTAL_CLEAN column missing!")

med_imd = merged_gdf_clean['IMD_quintile_london'].median()
med_ptal = merged_gdf_clean['MEAN_PTAL_CLEAN'].median()

merged_gdf_clean['high_deprivation'] = merged_gdf_clean['IMD_quintile_london'] > med_imd
merged_gdf_clean['high_connectivity'] = merged_gdf_clean['MEAN_PTAL_CLEAN'] > med_ptal

def assign_quadrant(row):
    if row['high_deprivation'] and row['high_connectivity']:
        return 'Transport Paradox'
    elif row['high_deprivation'] and not row['high_connectivity']:
        return 'Expected Deprivation'
    elif not row['high_deprivation'] and row['high_connectivity']:
        return 'Privileged Access'
    else:
        return 'Suburban Pattern'

merged_gdf_clean['quadrant'] = merged_gdf_clean.apply(assign_quadrant, axis=1).astype("category")


quadrant_stats = merged_gdf_clean['quadrant'].value_counts()
quadrant_pct = merged_gdf_clean['quadrant'].value_counts(normalize=True) * 100


for quad, count in quadrant_stats.items():
    pct = quadrant_pct[quad]
    print(f"{quad}: {count:,} ({pct:.1f}%)")


correlation = merged_gdf_clean['IMD_quintile_london'].corr(merged_gdf_clean['MEAN_PTAL_CLEAN'])

print(f"\nCorrelation analysis:")
print(f"Correlation coefficient between IMD quintile and PTAL: {correlation:.4f}")

if abs(correlation) < 0.1:
    print("-> Very weak correlation, confirming Centre for London's findings!")
elif abs(correlation) < 0.3:
    print("-> Weak correlation")
else:
    print("-> Moderate or strong correlation")



fig, ax = plt.subplots(1, 1, figsize=(15, 12))
colors = {
    'Transport Paradox': '#d62728',
    'Expected Deprivation': '#ff7f0e',
    'Privileged Access': '#2ca02c',
    'Suburban Pattern': '#1f77b4'
}

for quadrant, color in colors.items():
    subset = merged_gdf_clean[merged_gdf_clean['quadrant'] == quadrant]
    if len(subset) > 0:
        subset.plot(ax=ax, color=color, edgecolor='white', linewidth=0.1)

legend_handles = [
    mpatches.Patch(color=color, label=f"{quad} (n={len(merged_gdf_clean[merged_gdf_clean['quadrant']==quad]):,})")
    for quad, color in colors.items()
]

ax.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(0, 1))
ax.set_title("London Transport-Deprivation Spatial Pattern", fontsize=16, pad=20)
ax.axis('off')
plt.tight_layout()
plt.show()


print("\n=== Descriptive statistics of each quadrant ===")
quadrant_means = merged_gdf_clean.groupby('quadrant')[['MEAN_PTAL_CLEAN', 'IMD_quintile_london']]\
    .agg(['mean','std','min','max']).round(2)
print(quadrant_means)


In [ ]:
# =============================================================================
# Statistical Test: Kruskal–Wallis Analysis of PTAL and IMD across Quadrants
# =============================================================================

groups_ptal = [g['MEAN_PTAL_CLEAN'].dropna()
               for _, g in merged_gdf_clean.groupby('quadrant')]
H, p_kw = kruskal(*groups_ptal)
print('Kruskal-Wallis PTAL: H =', H, 'p =', p_kw)

groups_imd = [g['IMD_quintile_london'].dropna()
              for _, g in merged_gdf_clean.groupby('quadrant')]
H_imd, p_imd = kruskal(*groups_imd)
print('Kruskal-Wallis IMD: H =', H_imd, 'p =', p_imd)

In [ ]:
# =============================================================================
# Identification of Extreme Cases 
# =============================================================================

# Transport Paradox: Top 10 LSOAs with the highest connectivity
paradox_top = merged_gdf_clean[merged_gdf_clean['quadrant']=='Transport Paradox'].nlargest(10, 'MEAN_PTAL_CLEAN')
print("Top 10 LSOAs with the highest connectivity in the Transport Paradox quadrant:")
for i, (idx, row) in enumerate(paradox_top.iterrows(),1):
    print(f"{i:2d}. {row['LSOA21NM'][:30]:<30} PTAL:{row['MEAN_PTAL_CLEAN']:.1f} IMD:{row['IMD_quintile_london']}")

# Expected Deprivation: Bottom 10 LSOAs with the lowest connectivity
expected_worst = merged_gdf_clean[merged_gdf_clean['quadrant']=='Expected Deprivation'].nsmallest(10, 'MEAN_PTAL_CLEAN')
print("\nBottom 10 LSOAs with the lowest connectivity in the Expected Deprivation quadrant:")
for i, (idx, row) in enumerate(expected_worst.iterrows(),1):
    print(f"{i:2d}. {row['LSOA21NM'][:30]:<30} PTAL:{row['MEAN_PTAL_CLEAN']:.1f} IMD:{row['IMD_quintile_london']}")


In [ ]:
# =============================================================================
# Correlation Analysis: PTAL vs IMD Domains + Quadrant Deprivation Profiles (Radar Chart)
# =============================================================================

imd_domains = {
    "Income": "Income_london_decile",
    "Employment": "employment_london_decile",
    "Education": "edu_london_decile",
    "Health": "health_london_decile",
    "Crime": "crime_london_decile",
    "Housing": "barriers_london_decile",
    "Environment": "livingEnv_london_decile"
}


def interpret_correlation(r, p):
    if abs(r) < 0.1:
        strength = "Very weak"
    elif abs(r) < 0.3:
        strength = "Weak"
    elif abs(r) < 0.5:
        strength = "Moderate"
    else:
        strength = "Strong"
    direction = "positive" if r > 0 else "negative"
    if p < 0.05:
        return f"{strength} {direction}"
    else:
        return "Non-significant"

results = []
for domain, col in imd_domains.items():
    corr, pval = pearsonr(merged_gdf_clean['MEAN_PTAL_CLEAN'], merged_gdf_clean[col])
    results.append({
        "Domain": domain,
        "Correlation": round(corr,3),
        "P_value": "<0.001" if pval < 0.001 else f"{pval:.3f}",
        "Interpretation": interpret_correlation(corr, pval)
    })

correlation_results = pd.DataFrame(results)
print("\n=== Correlation results with Interpretation ===")
print(correlation_results.to_string(index=False))


if "quadrant" in merged_gdf_clean.columns:
    paradox_profile = merged_gdf_clean[merged_gdf_clean["quadrant"]=="Transport Paradox"][list(imd_domains.values())].mean()
    other_quadrants = merged_gdf_clean.groupby("quadrant")[list(imd_domains.values())].mean()

    print("\n=== Transport Paradox Profile ===")
    print(paradox_profile)

    print("\n=== Other Quadrants Profile ===")
    print(other_quadrants)



if "quadrant" in merged_gdf_clean.columns:
    labels = list(imd_domains.keys())
    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1] 

    fig, ax = plt.subplots(figsize=(7,7), subplot_kw=dict(polar=True))

    colors = {
        "Transport Paradox": "r",
        "Expected Deprivation": "b",
        "Privileged Access": "g",
        "Suburban Pattern": "orange"
    }

    for quad, row in other_quadrants.iterrows():
        vals = [row[col] for col in imd_domains.values()]
        vals += vals[:1]
        ax.plot(angles, vals, color=colors.get(quad, "gray"), linewidth=2, label=quad)
        ax.fill(angles, vals, color=colors.get(quad, "gray"), alpha=0.1)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_title("Deprivation Profiles by Quadrant")
    ax.legend(loc="upper right", bbox_to_anchor=(1.3,1.1))

    plt.show()




In [ ]:
# =============================================================================
# Function: Calculate Improved Policy Priority Score 
# (Normalized PTAL & IMD, Quadrant Weights, Composite Index, Priority Levels)
# =============================================================================

def calculate_improved_priority_score(df):
    """
    Improved policy priority calculation with enhanced differentiation and practicality
    """

    df = df.copy()
    
    # Normalize PTAL
    df['ptal_normalized'] = (df['MEAN_PTAL_CLEAN'] - df['MEAN_PTAL_CLEAN'].min()) / \
                           (df['MEAN_PTAL_CLEAN'].max() - df['MEAN_PTAL_CLEAN'].min())
    
    # Normalize IMD quintile (1–5 mapped to 0–1)
    df['imd_normalized'] = (df['IMD_quintile_london'] - 1) / 4  

    # Paradox intensity = high deprivation but low connectivity
    df['paradox_intensity'] = df['imd_normalized'] * (1 - df['ptal_normalized'])
    
    # Base quadrant weights
    quadrant_weights = {
        'Transport Paradox': 1.0,      # Highest priority
        'Expected Deprivation': 0.85,  # High priority
        'Suburban Pattern': 0.4,       # Medium priority
        'Privileged Access': 0.1       # Low priority
    }
    df['base_priority'] = df['quadrant'].map(quadrant_weights).astype(float)
    
    # Weighted score combining multiple factors, normalized to 0–1
    df['policy_priority_score'] = (
        df['base_priority'] * 3.0 +       # Quadrant weight
        df['imd_normalized'] * 2.0 +      # Level of deprivation
        (1 - df['ptal_normalized']) * 1.5 +  # Lack of transport accessibility
        df['paradox_intensity'] * 1.0     # Weighted paradox intensity
    ) / 7.5  
    
    # Rescale to 1–10
    df['policy_priority_score'] = df['policy_priority_score'] * 9 + 1
    
    # Assign categorical priority levels
    df['priority_level'] = pd.cut(
        df['policy_priority_score'], 
        bins=[0, 3, 5, 7, 8.5, 10], 
        labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
        include_lowest=True
    )
    
    return df



In [ ]:

merged_gdf_improved = calculate_improved_priority_score(merged_gdf_clean)

In [ ]:
# =============================================================================
# Figure: PTAL vs IMD Scatter Plot with Quadrant Classification and Trend Line
# =============================================================================

plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 12,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 14,
    'axes.linewidth': 1.2,
    'grid.alpha': 0.3,
    'axes.edgecolor': 'black'
})


# Figure 1: PTAL vs IMD Scatter Plot with Quadrant Analysis

def create_figure1_ptal_imd_scatter(data):

    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    
   
    colors_quad_academic = {
        'Transport Paradox': '#d62728',      
        'Expected Deprivation': '#ff7f0e',   
        'Privileged Access': '#2ca02c',      
        'Suburban Pattern': '#1f77b4'        
    }
    
    med_imd = data['IMD_quintile_london'].median()
    med_ptal = data['MEAN_PTAL_CLEAN'].median()
    

    for quad, color in colors_quad_academic.items():
        subset = data[data['quadrant'] == quad]
        if len(subset) > 0:
            if quad == 'Transport Paradox' and 'paradox_intensity' in subset.columns:
                scatter = ax.scatter(
                    subset['MEAN_PTAL_CLEAN'], 
                    subset['IMD_quintile_london'],
                    c=subset['paradox_intensity'],
                    cmap='Reds',
                    label=f'{quad} (n={len(subset):,})',
                    alpha=0.7, 
                    s=25, 
                    edgecolor='white', 
                    linewidth=0.3
                )
            else:
                scatter = ax.scatter(
                    subset['MEAN_PTAL_CLEAN'], 
                    subset['IMD_quintile_london'],
                    color=color,
                    label=f'{quad} (n={len(subset):,})',
                    alpha=0.7, 
                    s=25, 
                    edgecolor='white', 
                    linewidth=0.3
                )
    

    ax.axhline(y=med_imd, color='black', linestyle='--', alpha=0.7, linewidth=1.5,
               label=f'IMD Median = {med_imd:.1f}')
    ax.axvline(x=med_ptal, color='black', linestyle='--', alpha=0.7, linewidth=1.5,
               label=f'PTAL Median = {med_ptal:.1f}')

    ax.text(0.02, 0.98, 'Expected\nDeprivation', transform=ax.transAxes, 
            fontsize=10, ha='left', va='top', 
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    ax.text(0.98, 0.98, 'Transport\nParadox', transform=ax.transAxes, 
            fontsize=10, ha='right', va='top',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    ax.text(0.02, 0.02, 'Privileged\nAccess', transform=ax.transAxes, 
            fontsize=10, ha='left', va='bottom',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    ax.text(0.98, 0.02, 'Suburban\nPattern', transform=ax.transAxes, 
            fontsize=10, ha='right', va='bottom',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    

    X = data['MEAN_PTAL_CLEAN'].values.reshape(-1, 1)
    y = data['IMD_quintile_london'].values
    reg = LinearRegression().fit(X, y)
    trend_x = np.linspace(data['MEAN_PTAL_CLEAN'].min(), data['MEAN_PTAL_CLEAN'].max(), 100)
    trend_y = reg.predict(trend_x.reshape(-1, 1))
    ax.plot(trend_x, trend_y, 'k-', alpha=0.5, linewidth=2, 
            label=f'Trend (R² = {reg.score(X, y):.3f})')

    ax.set_xlabel('Public Transport Accessibility Level (PTAL)')
    ax.set_ylabel('Index of Multiple Deprivation Quintile (London)')
    ax.set_title('Transport-Deprivation Relationship in Greater London\n(Quadrant Classification)', 
                 pad=20)
    
    ax.set_xlim(0, data['MEAN_PTAL_CLEAN'].max() * 1.05)
    ax.set_ylim(0.8, 5.2)
    

    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    

    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, fancybox=True)
    

    ax.text(0.02, 0.88, f'Total LSOAs: {len(data):,}', transform=ax.transAxes,
            fontsize=10, bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    return fig


In [ ]:
# ============================================================================= 
# Figure: Policy Intervention Priority Index Map 
# =============================================================================

# Set serif font globally
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun']
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.unicode_minus'] = False

def create_figure2_priority_map(gdf_data):
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    
    colors_priority_academic = {
        'Very High': '#8B0000',
        'High': '#DC143C',
        'Medium': '#FF8C00',
        'Low': '#228B22',
        'Very Low': '#4682B4'
    }
    
    priority_order = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
    handles = []
    
    for level in priority_order:
        subset = gdf_data[gdf_data['priority_level'] == level]
        if len(subset) > 0:
            p = subset.plot(
                ax=ax, 
                color=colors_priority_academic[level], 
                label=f'{level} (n={len(subset):,})',
                edgecolor='white', 
                linewidth=0.1, 
                alpha=0.85
            )
            # manually collect handle for legend
            handles.append(plt.Line2D([0], [0], color=colors_priority_academic[level], 
                                      lw=6, label=f'{level} (n={len(subset):,})'))
    
    ax.set_title(
        'Spatial Distribution of Transport-Deprivation Policy Priority Index\nin Greater London', 
        fontsize=15, pad=25, weight='bold'
    )
    ax.set_axis_off()
    
    legend = ax.legend(
        handles=handles,
        title='Policy Priority Level',
        loc='upper left',
        bbox_to_anchor=(0.02, 0.98),
        frameon=True,
        fancybox=True,
        shadow=True,
        fontsize=11,
        title_fontsize=12
    )
    legend.get_frame().set_facecolor('white')
    legend.get_frame().set_alpha(0.95)
    
    # simple scale bar
    scale_bar = Rectangle(
        (0.02, 0.02), 0.15, 0.03, transform=ax.transAxes,
        facecolor='white', edgecolor='black', alpha=0.9
    )
    ax.add_patch(scale_bar)
    ax.text(
        0.095, 0.035, '10 km', transform=ax.transAxes, 
        ha='center', va='center', fontsize=9, weight='bold'
    )
    
    # north arrow
    ax.annotate(
        'N', xy=(0.95, 0.95), xytext=(0.95, 0.9),
        arrowprops=dict(arrowstyle='->', lw=2, color='black'),
        transform=ax.transAxes, ha='center', va='center',
        fontsize=14, weight='bold'
    )
    
    # data source
    ax.text(
        0.02, 0.06, 
        'Data: TfL PTAL 2021, MHCLG IMD 2019\nMethod: Multi-factor weighted analysis',
        transform=ax.transAxes, fontsize=9, 
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.9)
    )
    
    plt.tight_layout()
    return fig


In [ ]:
# =============================================================================
# Figure : Statistical Analysis Heatmap
# =============================================================================

def create_figure3_statistical_heatmap(data):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    crosstab = pd.crosstab(data['quadrant'], data['priority_level'])
    quadrant_order = ['Transport Paradox', 'Expected Deprivation', 'Suburban Pattern', 'Privileged Access']
    priority_order = ['Very High', 'High', 'Medium', 'Low', 'Very Low']
    crosstab = crosstab.reindex(index=quadrant_order, columns=priority_order, fill_value=0)
    
    sns.heatmap(crosstab, annot=True, fmt='d', cmap='YlOrRd', ax=ax1,
                cbar_kws={'label': 'Number of LSOAs'}, 
                linewidths=0.5, linecolor='white')
    ax1.set_title('Quadrant vs Priority Level Distribution', fontweight='bold', pad=15)
    ax1.set_xlabel('Policy Priority Level')
    ax1.set_ylabel('Transport-Deprivation Quadrant')
    
    chi2, p_value, dof, expected = stats.chi2_contingency(crosstab)
    ax1.text(0.02, 0.98, f'χ² = {chi2:.1f}, p < 0.001***', 
             transform=ax1.transAxes, fontsize=10, weight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    box_data = [data[data['quadrant'] == quad]['policy_priority_score'].dropna() 
                for quad in quadrant_order]
    bp = ax2.boxplot(box_data, labels=quadrant_order, patch_artist=True)
    
    colors = ['#d62728', '#ff7f0e', '#1f77b4', '#2ca02c']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax2.set_title('Policy Priority Score Distribution by Quadrant', fontweight='bold', pad=15)
    ax2.set_ylabel('Policy Priority Score (1-10)')
    ax2.set_xlabel('Transport-Deprivation Quadrant')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    f_stat, p_val = stats.f_oneway(*box_data)
    ax2.text(0.02, 0.98, f'ANOVA: F = {f_stat:.1f}, p < 0.001***', 
             transform=ax2.transAxes, fontsize=10, weight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    if 'paradox_intensity' in data.columns:
        paradox_data = data[data['quadrant'] == 'Transport Paradox']['paradox_intensity'].dropna()
        n, bins, patches = ax3.hist(paradox_data, bins=20, alpha=0.7, color='#d62728', 
                                   edgecolor='black', linewidth=0.8)
        
        mean_val = paradox_data.mean()
        std_val = paradox_data.std()
        ax3.axvline(mean_val, color='black', linestyle='--', linewidth=2,
                   label=f'Mean = {mean_val:.3f}')
        ax3.axvline(mean_val + std_val, color='gray', linestyle=':', linewidth=1.5,
                   label=f'±1 SD = {std_val:.3f}')
        ax3.axvline(mean_val - std_val, color='gray', linestyle=':', linewidth=1.5)
        
        ax3.set_title('Transport Paradox Intensity Distribution', fontweight='bold', pad=15)
        ax3.set_xlabel('Paradox Intensity Score (0-1)')
        ax3.set_ylabel('Number of LSOAs')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        shapiro_stat, shapiro_p = stats.shapiro(paradox_data.sample(min(5000, len(paradox_data))))
        ax3.text(0.98, 0.98, f'n = {len(paradox_data)}\nShapiro-Wilk: W = {shapiro_stat:.3f}', 
                 transform=ax3.transAxes, ha='right', va='top', fontsize=10,
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    priority_counts = data['priority_level'].value_counts().reindex(priority_order)
    colors_pie = ['#8B0000', '#DC143C', '#FF8C00', '#228B22', '#4682B4']
    
    wedges, texts, autotexts = ax4.pie(priority_counts.values, 
                                       labels=priority_counts.index,
                                       autopct='%1.1f%%',
                                       colors=colors_pie,
                                       startangle=90,
                                       textprops={'fontsize': 10})
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    ax4.set_title('Overall Priority Level Distribution', fontweight='bold', pad=15)
    
    total_lsoas = len(data)
    high_priority = priority_counts[['Very High', 'High']].sum()
    ax4.text(0.02, 0.02, f'Total LSOAs: {total_lsoas:,}\nHigh Priority: {high_priority:,} ({high_priority/total_lsoas*100:.1f}%)',
             transform=ax4.transAxes, fontsize=10,
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    return fig

In [ ]:
def generate_academic_figures(data, gdf_data):
    print("Generating academic figures...")
    
    print("Creating Figure 1: PTAL vs IMD Scatter Plot...")
    fig1 = create_figure1_ptal_imd_scatter(data)
    plt.show()
    
    print("Creating Figure 2: Policy Priority Index Map...")
    fig2 = create_figure2_priority_map(gdf_data)
    plt.show()
    
    print("Creating Figure 3: Statistical Analysis...")
    fig3 = create_figure3_statistical_heatmap(data)
    plt.show()
    
    return fig1, fig2, fig3

fig1, fig2, fig3 = generate_academic_figures(merged_gdf_improved, merged_gdf_improved)


In [ ]:
# =============================================================================
# Policy Priority Index Sensitivity Analysis 
# =============================================================================

def sensitivity_analysis_priority_index_fixed(data, gdf, n_iter=100, noise_level=0.1):
    baseline_quantiles = data['policy_priority_score'].quantile([0.2, 0.4, 0.6, 0.8]).values
    
    data['priority_baseline'] = pd.cut(
        data['policy_priority_score'],
        bins=[-np.inf, baseline_quantiles[0], baseline_quantiles[1], 
              baseline_quantiles[2], baseline_quantiles[3], np.inf],
        labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
    )
    
    changes_record = pd.DataFrame(0, index=data.index, columns=['change_count'])
    results = []

    for i in range(n_iter):
        perturbed_score = data['policy_priority_score'] * (
            1 + np.random.uniform(-noise_level, noise_level, size=len(data))
        )
        
        perturbed_priority = pd.cut(
            perturbed_score,
            bins=[-np.inf, baseline_quantiles[0], baseline_quantiles[1], 
                  baseline_quantiles[2], baseline_quantiles[3], np.inf],
            labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
        )
        
        perturbed_counts = perturbed_priority.value_counts(normalize=True).sort_index()
        results.append(perturbed_counts)
        
        changed = (perturbed_priority != data['priority_baseline'])
        changes_record['change_count'] += changed.astype(int)
    
    results_df = pd.DataFrame(results)
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    results_df.plot(kind='box', ax=ax[0], grid=True, patch_artist=True,
                    boxprops=dict(facecolor='#87CEEB', alpha=0.6))
    ax[0].set_title("Distribution of Priority Levels under Weight Perturbation", fontweight='bold')
    ax[0].set_ylabel("Proportion of LSOAs")
    
    base_counts = data['priority_baseline'].value_counts(normalize=True).sort_index()
    for i, level in enumerate(results_df.columns):
        ax[0].plot(i+1, base_counts[level], 'ro', label="Baseline" if i == 0 else "")
    ax[0].legend()
    
    sns.heatmap(results_df.T, cmap='YlOrRd', ax=ax[1], cbar_kws={'label': 'Proportion'})
    ax[1].set_title("Heatmap of Priority Distribution across Iterations", fontweight='bold')
    ax[1].set_xlabel("Iteration")
    ax[1].set_ylabel("Priority Level")
    
    plt.tight_layout()
    plt.show()
    
    print(results_df.describe().round(3))
    
    
    gdf['change_frequency'] = changes_record['change_count'] / n_iter  # 变化频率 (0-1)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    gdf.plot(column='change_frequency', cmap='Reds', legend=True, ax=ax,
             legend_kwds={'label': "Probability of Priority Change", 'shrink': 0.6},
             edgecolor='white', linewidth=0.1)
    
    ax.set_title("Spatial Sensitivity of Policy Priority Index\n(Change Frequency across Perturbations)",
                 fontsize=14, fontweight='bold', pad=15)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return results_df, gdf


sensitivity_results, merged_gdf_improved = sensitivity_analysis_priority_index_fixed(
    merged_gdf_improved, merged_gdf_improved, n_iter=200, noise_level=0.1
)


In [ ]:
# =============================================================================
# Figures: Global Moran’s I and LISA Cluster Maps (IMD Quintile & PTAL)
# =============================================================================

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun']
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid', rc={'font.family': 'serif'})

colors_quad_academic = {
   "High-High": "#b2182b",   
     "Low-Low":   "#2166ac",   
     "High-Low":  "#ef8a62",   
     "Low-High":  "#67a9cf",   
     "Not Significant": "#d9d9d9"
}


def build_weights(gdf, k_fallback=6):
    w = libpysal.weights.Queen.from_dataframe(gdf, ids=gdf.index.tolist())
    islands = w.islands
    if islands:
        coords = np.column_stack([gdf.geometry.centroid.x.values, gdf.geometry.centroid.y.values])
        w = libpysal.weights.KNN.from_array(coords, k=k_fallback, ids=gdf.index.tolist())
    w.transform = "r"
    return w

def moran_and_lisa_panels(gdf, value_cols, labels, permutations=999, alpha=0.05):
    fig, axes = plt.subplots(2, 1, figsize=(10, 14))

    results = {}
    for i, (value_col, var_label) in enumerate(zip(value_cols, labels)):
        data = gdf[[value_col, "geometry"]].dropna().copy().reset_index(drop=True)

        w = build_weights(data)
        y = data[value_col].values.astype(float)
        moran = Moran(y, w, permutations=permutations)
        lisa = Moran_Local(y, w, permutations=permutations)

        data["lisa_I"] = lisa.Is
        data["lisa_p"] = lisa.p_sim
        quad_map = {1: "High-High", 2: "Low-High", 3: "Low-Low", 4: "High-Low"}
        data["lisa_q"] = pd.Categorical([quad_map[q] for q in lisa.q], 
                                        categories=["High-High","Low-Low","High-Low","Low-High"], 
                                        ordered=False)
        data["lisa_sig"] = np.where(data["lisa_p"] < alpha, data["lisa_q"], "Not Significant")

        print(f"=== Global Moran's I ({var_label}) ===")
        print(f"I = {moran.I:.4f}, p-value = {moran.p_sim:.4f}, permutations = {permutations}")
        try:
            print(f"z-score ≈ {moran.z_norm:.3f}")
        except Exception:
            pass

        ax = axes[i]
        data.plot(ax=ax,
                  color=data["lisa_sig"].map(colors_quad_academic),
                  edgecolor="white", linewidth=0.1)
        ax.set_title(f"LISA Cluster Map ({var_label})", fontsize=15, pad=20)
        ax.axis("off")

        
        patches = [mpatches.Patch(color=colors_quad_academic[k], label=k) 
                   for k in ["High-High","Low-Low","High-Low","Low-High","Not Significant"]]
        ax.legend(handles=patches, loc="lower left", frameon=True)

        results[var_label] = {"gdf": data, "moran": moran, "lisa": lisa}

    plt.tight_layout(h_pad=3)
    plt.show()

    return results


results = moran_and_lisa_panels(
    merged_gdf_clean,
    ["IMD_quintile_london", "MEAN_PTAL_CLEAN"],
    ["IMD (Quintile)", "PTAL"]
)


In [ ]:
imd_values = merged_gdf_clean["IMD_quintile_london"].values

w = libpysal.weights.Queen.from_dataframe(merged_gdf_clean)
w.transform = "r"

lisa_imd = Moran_Local(imd_values, w)

merged_gdf_clean["IMD_LISA_I"] = lisa_imd.Is
merged_gdf_clean["IMD_LISA_p"] = lisa_imd.p_sim
merged_gdf_clean["IMD_LISA_cluster"] = lisa_imd.q

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
merged_gdf_clean.assign(
    cluster=merged_gdf_clean["IMD_LISA_cluster"].astype(str)
).plot(
    column="cluster", categorical=True, legend=True, cmap="Set1", linewidth=0.1, ax=ax
)
ax.set_title("Local Moran’s I Cluster Map (IMD)", fontsize=14)
ax.axis("off")
plt.show()


In [ ]:
imd_values = merged_gdf_clean['IMD_quintile_london'].values

w = libpysal.weights.Queen.from_dataframe(merged_gdf_clean)
w.transform = 'r'

lisa = Moran_Local(imd_values, w)

merged_gdf_clean['lisa_I'] = lisa.Is
merged_gdf_clean['lisa_p'] = lisa.p_sim
merged_gdf_clean['lisa_q'] = lisa.q

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
colors = {1: 'red', 2: 'lightblue', 3: 'blue', 4: 'pink'}
for q, color in colors.items():
    subset = merged_gdf_clean[(merged_gdf_clean['lisa_q'] == q) & (merged_gdf_clean['lisa_p'] < 0.05)]
    subset.plot(ax=ax, color=color, edgecolor='white', label=f'LISA Cluster {q} (p<0.05)')
ax.set_title("Local Moran's I (LISA) - IMD Hot/Cold Spots")
ax.axis('off')
ax.legend()
plt.show()


In [ ]:
# =============================================================================
# LISA–Housing Price Analysis: Join, Cluster Stats, and Paradox Identification
# =============================================================================

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun']
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid', rc={'font.family': 'serif'})

today = datetime.now().strftime("%Y%m%d")
save_dir = r"C:\Users\16643\Desktop\figures"
os.makedirs(save_dir, exist_ok=True)

print("Starting LISA housing price analysis...")

drop_cols = [c for c in merged_gdf_clean.columns if str(c).startswith('median_house_price')]
if drop_cols:
    merged_gdf_clean = merged_gdf_clean.drop(columns=drop_cols)

house_price_df = pd.read_csv(
    r"C:\Users\16643\Desktop\data\Median price paid for residential properties by LSOA.csv"
)

house_price_clean = (
    house_price_df[['LSOA code', 'Year ending Dec 2021']]
    .rename(columns={'LSOA code': 'LSOA11CD',
                     'Year ending Dec 2021': 'median_house_price'})
)

house_price_clean['median_house_price'] = (
    house_price_clean['median_house_price']
    .astype(str)
    .str.replace(',', '')
    .str.replace('£', '')
    .str.strip()
    .replace('', np.nan)
)

house_price_clean['median_house_price'] = pd.to_numeric(
    house_price_clean['median_house_price'], errors='coerce'
)
house_price_clean = house_price_clean.dropna()

house_price_2021 = (
    house_price_clean
    .merge(crosswalk_df, on='LSOA11CD', how='left')
    .dropna(subset=['LSOA21CD'])
    .groupby('LSOA21CD', as_index=False)['median_house_price']
    .median()
)

merged_gdf_clean = merged_gdf_clean.merge(
    house_price_2021[['LSOA21CD', 'median_house_price']],
    on='LSOA21CD',
    how='left'
)

significant_lisa = merged_gdf_clean.query("IMD_LISA_p < 0.05").copy()

cluster_labels = {
    1: 'High-High (Deprivation Hotspots)',
    2: 'Low-High (Outliers)', 
    3: 'Low-Low (Affluent Clusters)',
    4: 'High-Low (Outliers)'
}

significant_lisa['cluster_label'] = significant_lisa['IMD_LISA_cluster'].map(cluster_labels)

if significant_lisa['median_house_price'].notna().any():
    housing_stats = (
        significant_lisa
        .dropna(subset=['median_house_price'])
        .groupby('cluster_label')['median_house_price']
        .agg(['count', 'mean', 'median', 'std'])
        .round(0)
    )
    
    print("\nHousing price statistics by LISA clusters:")
    print(housing_stats.to_string(formatters={
        'mean': '£{:,.0f}'.format,
        'median': '£{:,.0f}'.format, 
        'std': '£{:,.0f}'.format
    }))

high_high_areas = significant_lisa[
    significant_lisa['cluster_label'] == 'High-High (Deprivation Hotspots)'
].copy()

if not high_high_areas.empty:
    ptal_column = None
    possible_ptal_names = ['MEAN_PTAL_CLEAN', 'MEAN_PTAL_', 'PTAL', 'mean_ptal']
    
    for col_name in possible_ptal_names:
        if col_name in high_high_areas.columns:
            ptal_column = col_name
            break
    
    if ptal_column is None:
        ptal_cols = [col for col in high_high_areas.columns if 'PTAL' in col.upper()]
        if ptal_cols:
            ptal_column = ptal_cols[0]
    
    if ptal_column and high_high_areas[[ptal_column, 'median_house_price']].dropna().shape[0] > 0:
        ptal_median = high_high_areas[ptal_column].median()
        price_q75 = high_high_areas['median_house_price'].quantile(0.75)
        
        high_high_areas['is_paradox'] = (
            (high_high_areas[ptal_column] >= ptal_median) &
            (high_high_areas['median_house_price'] >= price_q75)
        )
        
        paradox_areas = high_high_areas[high_high_areas['is_paradox']]
        
        print(f"\nTransport Paradox analysis results:")
        print(f"   Total High-High areas: {len(high_high_areas)}")
        print(f"   Paradox candidates: {len(paradox_areas)}")
        print(f"   Paradox proportion: {len(paradox_areas)/len(high_high_areas)*100:.1f}%")
        
        if not paradox_areas.empty:
            print(f"   Average house price (Paradox areas): £{paradox_areas['median_house_price'].mean():,.0f}")
            print(f"   Average house price (All High-High): £{high_high_areas['median_house_price'].mean():,.0f}")
            print(f"   Price ratio: {paradox_areas['median_house_price'].mean() / high_high_areas['median_house_price'].mean():.2f}x")



print("\nAnalysis Summary:")
print(f"   • Total LSOA areas: {len(merged_gdf_clean):,}")
print(f"   • With house price data: {merged_gdf_clean['median_house_price'].notna().sum():,}")
print(f"   • Significant LISA areas: {len(significant_lisa):,}")
print(f"   • High-High areas: {len(high_high_areas):,}")
if 'paradox_areas' in locals():
    print(f"   • Transport Paradox candidates: {len(paradox_areas):,}")
print(f"\nDefinition of Transport Paradox: High-High deprivation areas with both high transport accessibility and high house prices")



In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 12))

# LISA boxplot
if not significant_lisa.empty and significant_lisa['median_house_price'].notna().any():
    plot_data = significant_lisa.dropna(subset=['median_house_price', 'cluster_label'])
    sns.boxplot(
        data=plot_data,
        x='cluster_label',
        y='median_house_price',
        order=[label for label in cluster_labels.values() if label in plot_data['cluster_label'].values],
        ax=axes[0]
    )
    axes[0].set_title('Median House Prices across LISA Clusters', fontsize=15, pad=20)
    axes[0].set_xlabel('')
    axes[0].set_ylabel('Median House Price (£)')
    axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'£{x/1000:.0f}K'))

    new_labels = {
        'High-High (Deprivation Hotspots)': 'High-High\n(Deprivation Hotspots)',
        'Low-High (Outliers)': 'Low-High\n(Outliers)',
        'Low-Low (Affluent Clusters)': 'Low-Low\n(Affluent Clusters)',
        'High-Low (Outliers)': 'High-Low\n(Outliers)'
    }
    axes[0].set_xticklabels([new_labels.get(t.get_text(), t.get_text()) for t in axes[0].get_xticklabels()])
else:
    axes[0].text(0.5, 0.5, 'No Data Available', ha='center', va='center',
                 transform=axes[0].transAxes, fontsize=12)

# High-High scatter
if (not high_high_areas.empty and 'ptal_column' in locals() and 
    high_high_areas[[ptal_column, 'median_house_price']].dropna().shape[0] > 0):

    plot_data = high_high_areas.dropna(subset=[ptal_column, 'median_house_price'])
    colors = ['lightcoral' if not paradox else 'darkred' for paradox in plot_data['is_paradox']]

    axes[1].scatter(
        plot_data[ptal_column],
        plot_data['median_house_price'],
        c=colors, s=60, alpha=0.7, edgecolors='white', linewidth=0.5
    )
    axes[1].axhline(price_q75, color='gray', ls='--', lw=1, alpha=0.8,
                    label=f'Price Q75: £{price_q75:,.0f}')
    axes[1].axvline(ptal_median, color='gray', ls='--', lw=1, alpha=0.8,
                    label=f'PTAL Median: {ptal_median:.2f}')

    axes[1].text(0.02, 0.98, f'Transport Paradox:\n{len(paradox_areas)} areas',
                 transform=axes[1].transAxes, fontsize=10, verticalalignment='top',
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='darkred', alpha=0.1))

    axes[1].set_title('PTAL and House Prices in High-High IMD Areas',
                      fontsize=15, pad=20)
    axes[1].set_xlabel('PTAL Score (Transport Accessibility)')
    axes[1].set_ylabel('Median House Price (£)')
    axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'£{x/1000:.0f}K'))

    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightcoral',
                   markersize=8, label='Normal'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkred',
                   markersize=8, label='Paradox Candidate')
    ]
    axes[1].legend(
        handles=legend_elements,
        title='Area Type',
        loc='center left',
        bbox_to_anchor=(1, 0.5),
        frameon=True
    )
else:
    axes[1].text(0.5, 0.5, 'No High-High Areas\nwith Complete Data',
                 ha='center', va='center', transform=axes[1].transAxes, fontsize=12)

plt.tight_layout(h_pad=5)
plt.show()


In [ ]:
# --- setup / paths ---
today = datetime.now().strftime("%Y%m%d")
save_dir = r"C:\Users\16643\Desktop\figures"
os.makedirs(save_dir, exist_ok=True)

print("Starting LISA housing price analysis based on IMD quintile...")

# --- sanity checks for required columns ---
required_columns = ['IMD_quintile_london']
missing_cols = [col for col in required_columns if col not in merged_gdf_clean.columns]

if missing_cols:
    print(f"Missing required columns: {missing_cols}")
    print(f"Available columns: {[col for col in merged_gdf_clean.columns if 'IMD' in col or 'quintile' in col]}")
else:
    # --- input distribution overview ---
    print(f"Using IMD quintile data: {merged_gdf_clean['IMD_quintile_london'].value_counts().sort_index().to_dict()}")

    # --- build spatial weights and compute Local Moran's I ---
    print("Building spatial weights matrix...")
    w = libpysal.weights.Queen.from_dataframe(merged_gdf_clean)
    w.transform = 'r'
    print(f"Spatial weights: {w.n} areas, mean neighbors: {w.mean_neighbors:.2f}")

    print("Computing Local Moran's I...")
    imd_values = merged_gdf_clean['IMD_quintile_london'].values
    lisa = Moran_Local(imd_values, w)

    merged_gdf_clean['lisa_I'] = lisa.Is
    merged_gdf_clean['lisa_p'] = lisa.p_sim
    merged_gdf_clean['lisa_q'] = lisa.q

    print("LISA analysis completed")
    print(f"Significant areas (p<0.05): {(merged_gdf_clean['lisa_p'] < 0.05).sum()}/{len(merged_gdf_clean)}")

    cluster_counts = (
        merged_gdf_clean[merged_gdf_clean['lisa_p'] < 0.05]['lisa_q']
        .value_counts().sort_index()
    )
    print(f"Cluster distribution: {cluster_counts.to_dict()}")

    # --- house price processing ---
    print("\nProcessing house price data...")

    drop_cols = [c for c in merged_gdf_clean.columns if str(c).startswith('median_house_price')]
    if drop_cols:
        merged_gdf_clean = merged_gdf_clean.drop(columns=drop_cols)

    house_price_file = r"C:\Users\16643\Desktop\data\Median price paid for residential properties by LSOA.csv"

    if not os.path.exists(house_price_file):
        print(f"House price data file not found: {house_price_file}")
    else:
        house_price_df = pd.read_csv(house_price_file)

        house_price_clean = (
            house_price_df[['LSOA code', 'Year ending Dec 2021']]
            .rename(columns={'LSOA code': 'LSOA11CD',
                             'Year ending Dec 2021': 'median_house_price'})
        )

        house_price_clean['median_house_price'] = (
            house_price_clean['median_house_price']
            .astype(str)
            .str.replace(',', '')
            .str.replace('£', '')
            .str.strip()
            .replace('', np.nan)
        )

        house_price_clean['median_house_price'] = pd.to_numeric(
            house_price_clean['median_house_price'], errors='coerce'
        )
        house_price_clean = house_price_clean.dropna()

        house_price_2021 = (
            house_price_clean
            .merge(crosswalk_df, on='LSOA11CD', how='left')
            .dropna(subset=['LSOA21CD'])
            .groupby('LSOA21CD', as_index=False)['median_house_price']
            .median()
        )

        merged_gdf_clean = merged_gdf_clean.merge(
            house_price_2021[['LSOA21CD', 'median_house_price']],
            on='LSOA21CD',
            how='left'
        )

        price_coverage = merged_gdf_clean['median_house_price'].notna().sum()
        print(f"House price data merged - coverage: {price_coverage}/{len(merged_gdf_clean)} "
              f"({price_coverage/len(merged_gdf_clean)*100:.1f}%)")

        # --- LISA cluster housing price analysis ---
        print("\nLISA cluster housing price analysis...")

        significant_lisa = merged_gdf_clean[merged_gdf_clean['lisa_p'] < 0.05].copy()

        cluster_labels = {
            1: 'High-High (IMD Hotspots)',
            2: 'Low-High (Outliers)',
            3: 'Low-Low (Affluent Clusters)',
            4: 'High-Low (Outliers)'
        }

        significant_lisa['cluster_label'] = significant_lisa['lisa_q'].map(cluster_labels)

        if significant_lisa['median_house_price'].notna().any():
            housing_stats = (
                significant_lisa
                .dropna(subset=['median_house_price'])
                .groupby('cluster_label')['median_house_price']
                .agg(['count', 'mean', 'median', 'std'])
                .round(0)
            )

            print("Housing statistics by LISA clusters:")
            for cluster, stats in housing_stats.iterrows():
                print(f"   {cluster}:")
                print(f"     Count: {int(stats['count'])}")
                print(f"     Mean price: £{stats['mean']:,.0f}")
                print(f"     Median price: £{stats['median']:,.0f}")
                if not pd.isna(stats['std']):
                    print(f"     Std. dev.: £{stats['std']:,.0f}")

        # --- Transport Paradox analysis ---
        print("\nTransport Paradox analysis...")

        high_high_areas = significant_lisa[
            significant_lisa['cluster_label'] == 'High-High (IMD Hotspots)'
        ].copy()

        print(f"IMD deprivation hotspots: {len(high_high_areas)}")

        if not high_high_areas.empty:
            ptal_column = None
            possible_ptal_names = ['MEAN_PTAL_CLEAN', 'MEAN_PTAL_', 'PTAL', 'mean_ptal']

            for col_name in possible_ptal_names:
                if col_name in high_high_areas.columns:
                    ptal_column = col_name
                    break

            if ptal_column is None:
                ptal_cols = [col for col in high_high_areas.columns if 'PTAL' in col.upper()]
                if ptal_cols:
                    ptal_column = ptal_cols[0]

            print(f"Using PTAL column: {ptal_column}")

            if ptal_column and high_high_areas[[ptal_column, 'median_house_price']].dropna().shape[0] > 0:
                complete_data = high_high_areas.dropna(subset=[ptal_column, 'median_house_price'])
                print(f"Complete cases: {len(complete_data)}/{len(high_high_areas)}")

                ptal_median = complete_data[ptal_column].median()
                price_q75 = complete_data['median_house_price'].quantile(0.75)

                print(f"PTAL median: {ptal_median:.2f}")
                print(f"House price 75th percentile: £{price_q75:,.0f}")

                high_high_areas['is_paradox'] = (
                    (high_high_areas[ptal_column] >= ptal_median) &
                    (high_high_areas['median_house_price'] >= price_q75)
                )

                paradox_areas = high_high_areas[high_high_areas['is_paradox'] == True]

                print("\nTransport Paradox results:")
                print(f"Number of candidate areas: {len(paradox_areas)}")
                print(f"Share of hotspots: {len(paradox_areas)/len(high_high_areas)*100:.1f}%")

                if not paradox_areas.empty:
                    print(f"Average price (Paradox areas): £{paradox_areas['median_house_price'].mean():,.0f}")
                    print(f"Average price (All hotspots): £{complete_data['median_house_price'].mean():,.0f}")
                    print(f"Price ratio: {paradox_areas['median_house_price'].mean() / complete_data['median_house_price'].mean():.2f}x")
                    print(f"Average PTAL score: {paradox_areas[ptal_column].mean():.2f}")

# --- final textual summary ---
print("\n" + "="*60)
print("IMD-based LISA analysis summary")
print("="*60)
print(f"Total LSOA areas: {len(merged_gdf_clean):,}")

if 'significant_lisa' in locals():
    print(f"Significant LISA areas: {len(significant_lisa):,} "
          f"({len(significant_lisa)/len(merged_gdf_clean)*100:.1f}%)")

if 'price_coverage' in locals():
    print(f"Areas with house price data: {price_coverage:,} "
          f"({price_coverage/len(merged_gdf_clean)*100:.1f}%)")

if 'significant_lisa' in locals() and not significant_lisa.empty:
    cluster_labels = {
        1: 'High-High (IMD Hotspots)',
        2: 'Low-High (Outliers)',
        3: 'Low-Low (Affluent Clusters)',
        4: 'High-Low (Outliers)'
    }
    for cluster_id, label in cluster_labels.items():
        count = len(significant_lisa[significant_lisa['lisa_q'] == cluster_id])
        if count > 0:
            print(f"{label}: {count}")

if 'high_high_areas' in locals():
    print(f"\nIMD deprivation hotspots: {len(high_high_areas)}")
    if 'paradox_areas' in locals():
        print(f"Transport Paradox candidates: {len(paradox_areas)}")

print("\nTransport Paradox definition:")
print("   Within IMD deprivation hotspots, areas that simultaneously have:")
print("   • High transport accessibility (PTAL ≥ median)")
print("   • High house prices (≥ 75th percentile)")
print("="*60)


In [ ]:
merged_gdf_clean['Borough'] = merged_gdf_clean['LSOA21NM'].str[:6]  

formula = "IMD_quintile_london ~ MEAN_PTAL_CLEAN + C(Borough)"
ols_model = smf.ols(formula=formula, data=merged_gdf_clean).fit(cov_type='HC3')  
print(ols_model.summary())


In [ ]:
# =============================================================================
# Spatial Regression: OLS, Spatial Lag (SAR), and Spatial Error (SEM) — IMD ~ PTAL (Queen Weights)
# =============================================================================

y = merged_gdf_clean["IMD_quintile_london"].values.reshape(-1, 1)
X = merged_gdf_clean[["MEAN_PTAL_CLEAN"]].values

w = libpysal.weights.Queen.from_dataframe(merged_gdf_clean)
w.transform = "r"

ols_model = OLS(y, X, w=w, name_y="IMD", name_x=["PTAL"])
print("=== OLS Results ===")
print(ols_model.summary)

lag_model = ML_Lag(y, X, w=w, name_y="IMD", name_x=["PTAL"])
print("\n=== Spatial Lag Model Results ===")
print(lag_model.summary)

error_model = ML_Error(y, X, w=w, name_y="IMD", name_x=["PTAL"])
print("\n=== Spatial Error Model Results ===")
print(error_model.summary)


In [ ]:
def theil_index(x):
    x = np.array(x)
    x = x[x > 0]
    mean_x = np.mean(x)
    return np.sum((x / mean_x) * np.log(x / mean_x)) / len(x)

def atkinson_index(x, epsilon=0.5):
    x = np.array(x)
    x = x[x > 0]
    mean_x = np.mean(x)
    return 1 - (np.mean((x / mean_x) ** (1 - epsilon))) ** (1 / (1 - epsilon))

ineq_results = {}
for q in merged_gdf_clean['IMD_quintile_london'].unique():
    subset = merged_gdf_clean[merged_gdf_clean['IMD_quintile_london'] == q]
    ptal_vals = subset['MEAN_PTAL_CLEAN'].values
    ineq_results[q] = {
        "Theil": theil_index(ptal_vals),
        "Atkinson": atkinson_index(ptal_vals)
    }

for q, v in ineq_results.items():
    print(f"IMD Quintile {q}: Theil = {v['Theil']:.3f}, Atkinson = {v['Atkinson']:.3f}")


In [ ]:
# =============================================================================
# Analysis: Theil and Atkinson Indices of PTAL Distribution — 
# Bar/Line Plots across IMD Quintiles with Summary Table
# =============================================================================

data = {
    "IMD Quintile": [1, 2, 3, 4, 5],
    "Theil": [0.113, 0.113, 0.122, 0.136, 0.146],
    "Atkinson": [0.057, 0.057, 0.061, 0.068, 0.071]
}

df = pd.DataFrame(data)


fig, ax = plt.subplots(figsize=(8, 5))
bar_width = 0.35
x = range(len(df))

ax.bar([i - bar_width/2 for i in x], df["Theil"], width=bar_width, label="Theil Index")
ax.bar([i + bar_width/2 for i in x], df["Atkinson"], width=bar_width, label="Atkinson Index")

ax.set_xticks(x)
ax.set_xticklabels(df["IMD Quintile"])
ax.set_xlabel("IMD Quintile (1=Least deprived, 5=Most deprived)")
ax.set_ylabel("Inequality Index Value")
ax.set_title("Theil and Atkinson Indices of PTAL Distribution")
ax.legend()
plt.show()


fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(df["IMD Quintile"], df["Theil"], marker="o", label="Theil Index")
ax.plot(df["IMD Quintile"], df["Atkinson"], marker="s", label="Atkinson Index")

ax.set_xlabel("IMD Quintile (1=Least deprived, 5=Most deprived)")
ax.set_ylabel("Inequality Index Value")
ax.set_title("Trends of Transport Inequality across IMD Quintiles")
ax.legend()
plt.show()

print("=== Inequality indices by IMD Quintile ===")
print(df.to_string(index=False))


In [ ]:
# =========================
# Figure : Correlation between PTAL and IMD domains
# =========================

# Serif font settings
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Times', 'Georgia', 'STSong', 'SimSun']
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.unicode_minus'] = False

# Correlation data
correlation_data = {
    "Environment": -0.554,
    "Crime": -0.314,
    "Health": -0.250,
    "Income": -0.175,
    "Employment": -0.121,
    "Housing": -0.052,
    "Education": 0.087
}

# Sort by absolute correlation value
sorted_domains = sorted(correlation_data.items(), key=lambda x: abs(x[1]), reverse=True)
domains = [item[0] for item in sorted_domains]
correlations = [item[1] for item in sorted_domains]

# Colors: blue for negative, red for positive
colors = ['red' if corr > 0 else 'steelblue' for corr in correlations]

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(domains, correlations, color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)

# Labels on bars
for i, (domain, corr) in enumerate(zip(domains, correlations)):
    ax.text(corr + (0.01 if corr > 0 else -0.01), i, f'{corr:.3f}', 
            ha='left' if corr > 0 else 'right', va='center', fontweight='normal')

# Reference lines
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.axvline(x=-0.3, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=-0.5, color='gray', linestyle='--', alpha=0.5)

# Titles and labels
ax.set_title('Correlation between PTAL and IMD Domains', fontsize=15, pad=20)
ax.set_xlabel('Pearson Correlation Coefficient (r)', fontsize=12)
ax.set_ylabel('Deprivation Domain', fontsize=12)

# X-axis limits and ticks
ax.set_xlim(-0.6, 0.15)
ax.set_xticks(np.arange(-0.6, 0.2, 0.1))

# Grid
ax.grid(axis='x', alpha=0.3, linestyle='-', linewidth=0.5)

# Legend on the right
legend_elements = [
    Patch(facecolor='steelblue', alpha=0.7, label='Negative correlation'),
    Patch(facecolor='red', alpha=0.7, label='Positive correlation')
]
ax.legend(handles=legend_elements, title='Correlation', 
          loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=True)

# Note
plt.figtext(0.02, 0.02, 'Note: All correlations significant at p < 0.001', 
            fontsize=9, style='italic', ha='left')

plt.tight_layout()
plt.show()
